In [40]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

import shared_utils
import pandas as pd
import geopandas as gpd

import gcsfs
from calitp_data_analysis import get_fs
from calitp_data_analysis import geography_utils, utils
fs = get_fs()
import re
import google.auth
import os
import gcsfs
credentials, project = google.auth.default()
fs = gcsfs.GCSFileSystem()



In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

In [4]:
def read_parquet_from_gcs(filename):
    gcs_path = GCS_FILE_PATH.replace("gs://", "") + filename
    with fs.open(gcs_path, 'rb') as f:
        return gpd.read_parquet(f)

In [5]:
# start with trips per stop and ridership
stoptrips = read_parquet_from_gcs("tbl1_trips_perstop_07_08_2024.parquet")
stoptrips.head (5)

,name,location_type,route_type,stop_name,stop_id,stop_code,geometry,n_trips_weekday,n_trips_saturday,n_trips_sunday,n_routes_weekday,n_routes_saturday,n_routes_sunday,stop_desc
95,Monterey Salinas Schedule,0.00,3,Del Monte Center / Gate 1,0002,0002,POINT (-121.89736 36.58461),38.00,30.00,26.00,7.00,5.00,5.00,None
96,Monterey Salinas Schedule,0.00,3,Del Monte Center / Gate 2,0003,0003,POINT (-121.89728 36.58473),31.00,27.00,23.00,6.00,5.00,4.00,None
97,Monterey Salinas Schedule,0.00,3,Del Monte Center / Gate 3,0004,0004,POINT (-121.89893 36.58445),25.00,12.00,11.00,5.00,3.00,2.00,None
98,Monterey Salinas Schedule,0.00,3,6th / Mission Street,0006,0006,POINT (-121.92076 36.55564),91.00,86.00,77.00,6.00,6.00,6.00,None
99,Monterey Salinas Schedule,0.00,3,Northridge Mall,0011,0011,POINT (-121.65803 36.71563),142.00,136.00,132.00,6.00,7.00,7.00,None


In [6]:
ridership_metro = read_parquet_from_gcs("ridership_metro_08_26_2024.parquet")
ridership_metro.head(5)

,feed_key,stop_id,stop_name,geometry,STOP_NAME,sat_ons,sun_ons,weekday_ons,name
0,06d1f3ac2b0ae5e74424edbbfefa19ed,12591,LA Zoo,POINT (158199.490 -428414.858),LA ZOO,857.68,381.19,2775.55,LA Metro Bus Schedule
1,06d1f3ac2b0ae5e74424edbbfefa19ed,5377,1st / Hill,POINT (161833.578 -438634.619),1ST / HILL,14914.09,11531.02,159742.80,LA Metro Bus Schedule
2,06d1f3ac2b0ae5e74424edbbfefa19ed,15612,1st / Hill,POINT (161849.863 -438611.462),1ST / HILL,4800.62,3383.07,43551.06,LA Metro Bus Schedule
3,06d1f3ac2b0ae5e74424edbbfefa19ed,1217,6th / Wall,POINT (161822.154 -439849.592),6TH / WALL,3502.19,3061.44,24682.11,LA Metro Bus Schedule
4,06d1f3ac2b0ae5e74424edbbfefa19ed,7376,7th / Alma,POINT (157649.168 -473829.124),7TH / ALMA,0.00,35.74,559.87,LA Metro Bus Schedule


In [7]:
ridership_sbmtd = read_parquet_from_gcs("ridership_sbmtd_08_26_2024.parquet")
ridership_sbmtd.head(5)

,feed_key,stop_id,stop_name,geometry,sat_ons,sun_ons,weekday_ons,name
0,52201caab047b98ae19b7547c0d7c2ad,1,Modoc & Portesuello,POINT (25170.737 -398993.625),1573.00,1287.00,13964.00,SBMTD Schedule
1,52201caab047b98ae19b7547c0d7c2ad,2,Milpas & Montecito,POINT (29196.552 -399052.308),3901.00,3139.00,30225.00,SBMTD Schedule
2,52201caab047b98ae19b7547c0d7c2ad,4,Cathedral Oaks & Camino Del Rio,POINT (20247.563 -395908.292),NaN,NaN,1.00,SBMTD Schedule
3,52201caab047b98ae19b7547c0d7c2ad,5,Via Real & Sandpiper MHP,POINT (42143.060 -400995.911),217.00,109.00,1485.00,SBMTD Schedule
4,52201caab047b98ae19b7547c0d7c2ad,6,UCSB Elings Hall Outbound,POINT (14738.802 -400125.683),1374.00,1199.00,9777.00,SBMTD Schedule


In [8]:
ridership_mst = read_parquet_from_gcs("ridership_mst_08_26_2024.parquet")
ridership_mst.head(5)

,feed_key,stop_id,stop_name,geometry,sat_ons,sun_ons,weekday_ons,name
0,118c3a62eab691ac449fe0c1c7505413,0002,Del Monte Center / Gate 1,POINT (-169532.243 -157455.258),2240.00,1740.00,9287.00,Monterey Salinas Schedule
1,118c3a62eab691ac449fe0c1c7505413,0003,Del Monte Center / Gate 2,POINT (-169524.827 -157441.956),616.00,348.00,1757.00,Monterey Salinas Schedule
2,118c3a62eab691ac449fe0c1c7505413,0004,Del Monte Center / Gate 3,POINT (-169673.036 -157470.002),56.00,58.00,502.00,Monterey Salinas Schedule
3,118c3a62eab691ac449fe0c1c7505413,0006,6th / Mission Street,POINT (-171687.391 -160632.627),4312.00,4234.00,21837.00,Monterey Salinas Schedule
4,118c3a62eab691ac449fe0c1c7505413,0011,Northridge Mall,POINT (-147895.267 -143297.134),9128.00,13224.00,40662.00,Monterey Salinas Schedule


In [9]:
# contatenate gdfs, keeping common columns
ridership_all = pd.concat([ridership_metro,ridership_sbmtd,ridership_mst], join='inner', ignore_index="True")
ridership_all.sample(5)

,feed_key,stop_id,stop_name,geometry,sat_ons,sun_ons,weekday_ons,name
11561,06d1f3ac2b0ae5e74424edbbfefa19ed,4723,Temple / Bonnie Brae,POINT (160082.174 -436981.190),810.03,702.82,8398.11,LA Metro Bus Schedule
4331,06d1f3ac2b0ae5e74424edbbfefa19ed,16392,Valley / New,POINT (174593.633 -435610.317),1596.24,1500.94,14425.69,LA Metro Bus Schedule
1332,06d1f3ac2b0ae5e74424edbbfefa19ed,15159,Van Nuys / Plummer,POINT (142716.437 -418109.146),5205.64,4133.54,42371.75,LA Metro Bus Schedule
2123,06d1f3ac2b0ae5e74424edbbfefa19ed,146,Nash / Mariposa Station,POINT (149104.326 -453414.919),690.91,369.28,10304.07,LA Metro Bus Schedule
10736,06d1f3ac2b0ae5e74424edbbfefa19ed,142000,Santa Ana / State,POINT (165376.302 -448668.328),142.95,95.30,690.91,LA Metro Bus Schedule


In [10]:
stoptrips.columns

Index(['name', 'location_type', 'route_type', 'stop_name', 'stop_id',
       'stop_code', 'geometry', 'n_trips_weekday', 'n_trips_saturday',
       'n_trips_sunday', 'n_routes_weekday', 'n_routes_saturday',
       'n_routes_sunday', 'stop_desc'],
      dtype='object')

In [11]:
len(ridership_all)

13677

In [12]:
len(stoptrips)

13730

In [13]:
trips_ridership_joined = pd.merge(
    stoptrips,
    ridership_all[['feed_key', 'name', 'stop_id', 'stop_name', 'sat_ons', 'sun_ons', 'weekday_ons']],
    on=['name', 'stop_id', 'stop_name'],
    how='outer'
)


In [14]:
len(trips_ridership_joined)

13873

In [15]:
#Rename 'geometry_x' to 'geometry'
trips_ridership_joined = trips_ridership_joined.rename(columns={'geometry_x': 'geometry'})

#Set 'geometry' as the active geometry column
trips_ridership_joined = trips_ridership_joined.set_geometry('geometry')

In [16]:
trips_ridership_joined = trips_ridership_joined.to_crs(epsg=3347)

In [17]:
stop_geom = trips_ridership_joined[["feed_key", "stop_id", "geometry"]]

In [18]:
trips_ridership_joined.geometry = trips_ridership_joined.buffer(402.336)

In [19]:
acs_ca = read_parquet_from_gcs("tracts_ca_acs.parquet")
len(acs_ca)

9109

In [20]:
len(acs_ca)

9109

In [21]:
# Load job density data from GCS and select required columns
jobdata = pd.read_parquet(f"{GCS_FILE_PATH}/job_density_2022.parquet")[['GEOID', 'jobs_tot']]

In [22]:
jobdata.columns

Index(['GEOID', 'jobs_tot'], dtype='object')

In [23]:
jobdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9103 entries, 0 to 9102
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   GEOID     9103 non-null   object
 1   jobs_tot  9103 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 142.4+ KB


In [24]:
jobdata.head(5)

,GEOID,jobs_tot
0,06001400100,285
1,06001400200,1156
2,06001400300,2022
3,06001400400,1036
4,06001400500,348


In [25]:
# Inner join with ACS data on 'geo_id'
acs_ca = acs_ca.merge(jobdata, on = 'GEOID', how='left')

In [26]:
acs_ca.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 9109 entries, 0 to 9108
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   ALAND                          9109 non-null   int64   
 1   geometry                       9109 non-null   geometry
 2   GEOID                          9109 non-null   object  
 3   total_pop                      9109 non-null   int64   
 4   households                     9109 non-null   int64   
 5   not_us_citizen_pop             9109 non-null   int64   
 6   black_pop                      9109 non-null   int64   
 7   hispanic_pop                   9109 non-null   int64   
 8   inc_extremelylow               9109 non-null   int64   
 9   inc_verylow                    9109 non-null   int64   
 10  inc_low                        9109 non-null   int64   
 11  poverty                        9109 non-null   int64   
 12  pop_determined_poverty_sta

In [27]:
acs_ca = acs_ca.to_crs(epsg=3347)

In [28]:
stops_acs_joined = trips_ridership_joined.sjoin(acs_ca, how='left', predicate='intersects')

In [29]:
len(trips_ridership_joined)

13873

In [30]:
len(stops_acs_joined)

44673

In [31]:
stops_acs_joined_count = stops_acs_joined.groupby(['feed_key', 'stop_id']).size().reset_index(name='count')
stops_acs_joined_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13535 entries, 0 to 13534
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   feed_key  13535 non-null  object
 1   stop_id   13535 non-null  object
 2   count     13535 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 317.4+ KB


In [32]:
stops_acs_rollup = (
    stops_acs_joined
    .groupby([
        'feed_key', 'stop_id', 'stop_name',
        'n_trips_weekday', 'n_trips_saturday', 'n_trips_sunday',
        'n_routes_weekday', 'n_routes_saturday', 'n_routes_sunday',
        'sat_ons', 'sun_ons', 'weekday_ons'
    ])
    .agg(
        sum_tracts=('GEOID', pd.Series.nunique),
        sum_total_pop=('total_pop', 'sum'),
        sum_households=('households', 'sum'),
        sum_not_us_citizen_pop=('not_us_citizen_pop', 'sum'),
        sum_black_pop=('black_pop', 'sum'),
        sum_hispanic_pop=('hispanic_pop', 'sum'),
        sum_youth_pop=('youth_pop', 'sum'),
        sum_seniors_pop=('seniors_pop', 'sum'),
        sum_inc_extremelylow=('inc_extremelylow', 'sum'),
        sum_inc_verylow=('inc_verylow', 'sum'),
        sum_inc_low=('inc_low', 'sum'),
        sum_pop_determined_poverty_status=('pop_determined_poverty_status', 'sum'),
        sum_poverty=('poverty', 'sum'),
        sum_no_car=('workers_with_no_car', 'sum'),
        sum_no_cars=('households_with_no_cars', 'sum'),
        sum_land_area=('ALAND', 'sum'),
        sum_jobs=('jobs_tot', 'sum')
    )
    .reset_index()
)

# Display the first 5 rows
stops_acs_rollup.head(5)


,feed_key,stop_id,stop_name,n_trips_weekday,n_trips_saturday,n_trips_sunday,n_routes_weekday,n_routes_saturday,n_routes_sunday,sat_ons,sun_ons,weekday_ons,sum_tracts,sum_total_pop,sum_households,sum_not_us_citizen_pop,sum_black_pop,sum_hispanic_pop,sum_youth_pop,sum_seniors_pop,sum_inc_extremelylow,sum_inc_verylow,sum_inc_low,sum_pop_determined_poverty_status,sum_poverty,sum_no_car,sum_no_cars,sum_land_area,sum_jobs
0,06d1f3ac2b0ae5e74424edbbfefa19ed,1,Paramount / Slauson,61.00,55.00,55.00,2.00,2.00,2.00,190.60,285.89,2549.21,1,4255.00,1080.00,479.00,21.00,4043.00,1535.00,584.00,1077.00,630.00,472.00,4048.00,481.00,30.00,72.00,3707186.00,7985.00
1,06d1f3ac2b0ae5e74424edbbfefa19ed,10000002,Artesia / Downey,24.00,15.00,15.00,1.00,1.00,1.00,238.24,83.39,1215.05,4,16169.00,3834.00,2863.00,1917.00,10586.00,5642.00,1500.00,4175.00,3445.00,1382.00,15804.00,1713.00,247.00,323.00,3244426.00,3961.00
2,06d1f3ac2b0ae5e74424edbbfefa19ed,10033,Hyde Park / Gay,50.00,30.00,30.00,1.00,1.00,1.00,262.07,285.89,2442.00,3,13380.00,2519.00,1998.00,5552.00,5950.00,3832.00,2122.00,4238.00,2280.00,1315.00,13224.00,2069.00,371.00,1021.00,2711237.00,1153.00
3,06d1f3ac2b0ae5e74424edbbfefa19ed,10034,Hyde Park / Long,50.00,30.00,30.00,1.00,1.00,1.00,83.39,59.56,1572.41,4,17267.00,3479.00,2220.00,7960.00,6902.00,5167.00,2837.00,5570.00,2821.00,1752.00,17111.00,3112.00,464.00,1399.00,3562791.00,1732.00
4,06d1f3ac2b0ae5e74424edbbfefa19ed,10035,Hyde Park / Park,50.00,30.00,30.00,1.00,1.00,1.00,452.66,405.02,4419.43,3,14737.00,3074.00,2366.00,5022.00,7789.00,4587.00,1925.00,3859.00,3454.00,1596.00,14577.00,1486.00,240.00,368.00,2565582.00,1805.00


In [33]:
def add_acs_metrics(df):
    df = df.copy()

    # Basic area conversion
    df['land_area_sqkm'] = df['sum_land_area'] / 1_000_000

    # Densities
    df['pop_density'] = df['sum_total_pop'] / df['land_area_sqkm']
    df['job_density'] = df['sum_jobs'] / df['land_area_sqkm']

    # Population percentages
    df['pct_not_us_citizen_pop'] = df['sum_not_us_citizen_pop'] / df['sum_total_pop']
    df['pct_black_pop'] = df['sum_black_pop'] / df['sum_total_pop']
    df['pct_hispanic_pop'] = df['sum_hispanic_pop'] / df['sum_total_pop']
    df['pct_youth_pop'] = df['sum_youth_pop'] / df['sum_total_pop']
    df['pct_seniors_pop'] = df['sum_seniors_pop'] / df['sum_total_pop']

    # Income levels
    df['pct_inc_extremelylow'] = df['sum_inc_extremelylow'] / df['sum_households']
    df['pct_inc_verylow'] = df['sum_inc_verylow'] / df['sum_households']
    df['pct_inc_low'] = df['sum_inc_low'] / df['sum_households']

    # Poverty & transportation
    df['pct_poverty'] = df['sum_poverty'] / df['sum_pop_determined_poverty_status']
    df['pct_pop_workers_no_car'] = df['sum_no_car'] / df['sum_total_pop']
    df['pct_hh_no_cars'] = df['sum_no_car'] / df['sum_households']

    return df


In [34]:
stops_acs_rollup = add_acs_metrics(stops_acs_rollup)

In [35]:
stops_acs_rollup.head(5)

,feed_key,stop_id,stop_name,n_trips_weekday,n_trips_saturday,n_trips_sunday,n_routes_weekday,n_routes_saturday,n_routes_sunday,sat_ons,sun_ons,weekday_ons,sum_tracts,sum_total_pop,sum_households,sum_not_us_citizen_pop,sum_black_pop,sum_hispanic_pop,sum_youth_pop,sum_seniors_pop,sum_inc_extremelylow,sum_inc_verylow,sum_inc_low,sum_pop_determined_poverty_status,sum_poverty,sum_no_car,sum_no_cars,sum_land_area,sum_jobs,land_area_sqkm,pop_density,job_density,pct_not_us_citizen_pop,pct_black_pop,pct_hispanic_pop,pct_youth_pop,pct_seniors_pop,pct_inc_extremelylow,pct_inc_verylow,pct_inc_low,pct_poverty,pct_pop_workers_no_car,pct_hh_no_cars
0,06d1f3ac2b0ae5e74424edbbfefa19ed,1,Paramount / Slauson,61.00,55.00,55.00,2.00,2.00,2.00,190.60,285.89,2549.21,1,4255.00,1080.00,479.00,21.00,4043.00,1535.00,584.00,1077.00,630.00,472.00,4048.00,481.00,30.00,72.00,3707186.00,7985.00,3.71,1147.77,2153.92,0.11,0.00,0.95,0.36,0.14,1.00,0.58,0.44,0.12,0.01,0.03
1,06d1f3ac2b0ae5e74424edbbfefa19ed,10000002,Artesia / Downey,24.00,15.00,15.00,1.00,1.00,1.00,238.24,83.39,1215.05,4,16169.00,3834.00,2863.00,1917.00,10586.00,5642.00,1500.00,4175.00,3445.00,1382.00,15804.00,1713.00,247.00,323.00,3244426.00,3961.00,3.24,4983.62,1220.86,0.18,0.12,0.65,0.35,0.09,1.09,0.90,0.36,0.11,0.02,0.06
2,06d1f3ac2b0ae5e74424edbbfefa19ed,10033,Hyde Park / Gay,50.00,30.00,30.00,1.00,1.00,1.00,262.07,285.89,2442.00,3,13380.00,2519.00,1998.00,5552.00,5950.00,3832.00,2122.00,4238.00,2280.00,1315.00,13224.00,2069.00,371.00,1021.00,2711237.00,1153.00,2.71,4935.02,425.27,0.15,0.41,0.44,0.29,0.16,1.68,0.91,0.52,0.16,0.03,0.15
3,06d1f3ac2b0ae5e74424edbbfefa19ed,10034,Hyde Park / Long,50.00,30.00,30.00,1.00,1.00,1.00,83.39,59.56,1572.41,4,17267.00,3479.00,2220.00,7960.00,6902.00,5167.00,2837.00,5570.00,2821.00,1752.00,17111.00,3112.00,464.00,1399.00,3562791.00,1732.00,3.56,4846.48,486.14,0.13,0.46,0.40,0.30,0.16,1.60,0.81,0.50,0.18,0.03,0.13
4,06d1f3ac2b0ae5e74424edbbfefa19ed,10035,Hyde Park / Park,50.00,30.00,30.00,1.00,1.00,1.00,452.66,405.02,4419.43,3,14737.00,3074.00,2366.00,5022.00,7789.00,4587.00,1925.00,3859.00,3454.00,1596.00,14577.00,1486.00,240.00,368.00,2565582.00,1805.00,2.57,5744.12,703.54,0.16,0.34,0.53,0.31,0.13,1.26,1.12,0.52,0.10,0.02,0.08


In [36]:
stops_acs_rollup_geom = stop_geom.merge(stops_acs_rollup, how='left', on=['stop_id', 'feed_key'])

In [37]:
stops_acs_rollup_geom.head(5)

,feed_key,stop_id,geometry,stop_name,n_trips_weekday,n_trips_saturday,n_trips_sunday,n_routes_weekday,n_routes_saturday,n_routes_sunday,sat_ons,sun_ons,weekday_ons,sum_tracts,sum_total_pop,sum_households,sum_not_us_citizen_pop,sum_black_pop,sum_hispanic_pop,sum_youth_pop,sum_seniors_pop,sum_inc_extremelylow,sum_inc_verylow,sum_inc_low,sum_pop_determined_poverty_status,sum_poverty,sum_no_car,sum_no_cars,sum_land_area,sum_jobs,land_area_sqkm,pop_density,job_density,pct_not_us_citizen_pop,pct_black_pop,pct_hispanic_pop,pct_youth_pop,pct_seniors_pop,pct_inc_extremelylow,pct_inc_verylow,pct_inc_low,pct_poverty,pct_pop_workers_no_car,pct_hh_no_cars
0,118c3a62eab691ac449fe0c1c7505413,0002,POINT (3437038.068 672539.232),Del Monte Center / Gate 1,38.00,30.00,26.00,7.00,5.00,5.00,2240.00,1740.00,9287.00,3.00,12519.00,1241.00,578.00,216.00,2087.00,2025.00,4303.00,2100.00,1593.00,1673.00,12198.00,881.00,101.00,389.00,55893539.00,12555.00,55.89,223.98,224.62,0.05,0.02,0.17,0.16,0.34,1.69,1.28,1.35,0.07,0.01,0.08
1,118c3a62eab691ac449fe0c1c7505413,0003,POINT (3437051.391 672548.520),Del Monte Center / Gate 2,31.00,27.00,23.00,6.00,5.00,4.00,616.00,348.00,1757.00,3.00,12519.00,1241.00,578.00,216.00,2087.00,2025.00,4303.00,2100.00,1593.00,1673.00,12198.00,881.00,101.00,389.00,55893539.00,12555.00,55.89,223.98,224.62,0.05,0.02,0.17,0.16,0.34,1.69,1.28,1.35,0.07,0.01,0.08
2,118c3a62eab691ac449fe0c1c7505413,0004,POINT (3436895.837 672590.854),Del Monte Center / Gate 3,25.00,12.00,11.00,5.00,3.00,2.00,56.00,58.00,502.00,3.00,12519.00,1241.00,578.00,216.00,2087.00,2025.00,4303.00,2100.00,1593.00,1673.00,12198.00,881.00,101.00,389.00,55893539.00,12555.00,55.89,223.98,224.62,0.05,0.02,0.17,0.16,0.34,1.69,1.28,1.35,0.07,0.01,0.08
3,118c3a62eab691ac449fe0c1c7505413,0006,POINT (3433486.450 670499.928),6th / Mission Street,91.00,86.00,77.00,6.00,6.00,6.00,4312.00,4234.00,21837.00,2.00,3186.00,139.00,246.00,29.00,122.00,147.00,1870.00,461.00,628.00,535.00,3184.00,155.00,47.00,111.00,2747956.00,2572.00,2.75,1159.41,935.97,0.08,0.01,0.04,0.05,0.59,3.32,4.52,3.85,0.05,0.01,0.34
4,118c3a62eab691ac449fe0c1c7505413,0011,POINT (3464418.138 676021.248),Northridge Mall,142.00,136.00,132.00,6.00,7.00,7.00,9128.00,13224.00,40662.00,3.00,15642.00,4793.00,4421.00,315.00,13109.00,6723.00,1774.00,3580.00,3006.00,1563.00,15615.00,1866.00,47.00,289.00,31903783.00,5849.00,31.90,490.29,183.33,0.28,0.02,0.84,0.43,0.11,0.75,0.63,0.33,0.12,0.00,0.01


In [38]:
def export_gdf(gdf, filename: str):
    
    gdf.to_parquet(f"{filename}.parquet")
    
    fs.put(
        f"{filename}.parquet",
        f"{GCS_FILE_PATH}/{filename}.parquet",
        token = credentials.token
    )
    
    os.remove(f"{filename}.parquet")
    print(f"saved {GCS_FILE_PATH}/{filename}.parquet")
    
    return

In [41]:
export_gdf(stops_acs_rollup_geom, "analytical_tbl.parquet")

saved gs://calitp-analytics-data/data-analyses/ahsc_grant//analytical_tbl.parquet.parquet
